In [ ]:
"""
tournaments.ipynb

Jupyter Notebook to perform analysis on card market data.

Author: Casey Forey
Date Created: 4/7/24
"""

In [ ]:
# Imports
import json 
import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import src.load.load_tournament_data as ltd
import src.plot.plot_tournament_data as ptd

In [ ]:
# Constants
DATA_DIRECTORY: str = 'data/'

In [ ]:
path = 'data/AtomicCards.json' 
f = open(path, encoding= "utf8") 
atm_cards = json.load(f)

path = 'data/AllPrintings.json' 
f = open(path, encoding= "utf8") 
all_cards = json.load(f)

path = 'data/AllPricesToday.json' 
f = open(path) 
prices = json.load(f)

path = 'data/SetList.json' 
f = open(path, encoding= "utf8") 
set_list = json.load(f)

## Modern Data

In [ ]:
base_path: str = os.path.join(DATA_DIRECTORY, '2023')
modern_card_counts_df, modern_player_counts_df= ltd.load_format_card_counts(base_path, 'modern')
modern_card_counts_df

In [ ]:
data = atm_cards['data']
cards_set = {}
i = 0
for card in data:
    card_info = data[card][0]
    if 'firstPrinting' in card_info:
        cards_set[card] = card_info['firstPrinting']
    else:
        i +=1
print(i)

In [ ]:
cards_set_df = pd.DataFrame.from_dict(cards_set, orient='index')
cards_set_df.reset_index(inplace= True) 
cards_set_df.rename(columns={'index': 'card_name', 0:'set_name'},inplace= True) 
cards_set_df

In [ ]:
def add_back_face(df):
    for card in df['card_name']:
        for name in cards_set_df['card_name']:
            front_name = name.split('//')[0].strip()
            if card == front_name and '//' in name:
                df['card_name'].replace(card, name,inplace = True)
    return df

In [ ]:
modern_card_counts_df = add_back_face(modern_card_counts_df)

In [ ]:
# Checking to see if double face cards look correct
for name in modern_card_counts_df['card_name']:
    if "Smelt" in name:
        print(name)

In [ ]:
modern_card_counts_df[modern_card_counts_df['card_name'] == 'Smelt // Herd // Saw']

In [ ]:
modern_card_counts_df = modern_card_counts_df.merge(cards_set_df, on = 'card_name', how = 'left')
modern_card_counts_df

In [ ]:
set_year = {}
for set_id in set_list['data']:
    set_year[set_id['code']] = set_id['releaseDate'][0:4]
set_year_df = pd.DataFrame.from_dict(set_year, orient='index')
set_year_df.reset_index(inplace= True) 
set_year_df.rename(columns={'index': 'set_name', 0:'set_year'},inplace= True) 
set_year_df

In [ ]:
modern_card_counts_df = modern_card_counts_df.merge(set_year_df, on = 'set_name', how = 'left')

In [ ]:
# Grouping by 'set_year' and 'set_name', and summing up 'total_count'
grouped_data = modern_card_counts_df.groupby(['set_year','set_name'])['total_count'].sum().unstack().fillna(0)

# Creating a DataFrame with 'set_name' as a column
sets = pd.DataFrame({'set_name': grouped_data.columns})

# Merging with 'set_year_df' to get the release year
sets = pd.merge(sets, set_year_df, on='set_name', how='left')
sets

# Plotting the data as a stacked bar chart
ax = grouped_data.plot(kind='bar', stacked=True)

# Title and axis labels
plt.title('Card Counts by Set Name and Year')
plt.xlabel('Year')
plt.ylabel('Total Card Count')

# Modifying legend to include the year in set name and order by set name
handles, labels = ax.get_legend_handles_labels()
new_labels = [f'{label} - {year}' for label, year in zip(sets['set_name'], sets['set_year'])]
ax.legend(handles, new_labels, title='Set Name', loc='upper left', bbox_to_anchor=(1, 1))

plt.show()

In [ ]:
# Very limited Graph as a proof of concept
modern_card_counts_df.groupby('set_year')['total_count'].sum().plot(kind = 'bar')

In [ ]:
plt = ptd.create_base_graph(modern_card_counts_df,(255/255,122/255,14/255))
plt.axvline(x=10.0, color='blue', linestyle='--')
plt.text(10.2, 100000, 'Modern Legal', color = 'black', fontsize = 10)
plt.axvline(x=25.5, color='black', linestyle='--')
plt.text(22, 130000, 'Fire Design', color = 'black', fontsize = 10)
plt.axvline(x=28, color='purple', linestyle='--')
plt.text(28.4, 130000, 'MH2', color = 'black', fontsize = 10)
plt.margins(x=0)

### Pioneer Data

In [ ]:
base_path: str = os.path.join(DATA_DIRECTORY, '2023')
pioneer_card_counts_df, pioneer_player_counts_df = ltd.load_format_card_counts(base_path, 'pioneer')
pioneer_card_counts_df

In [ ]:
pioneer_card_counts_df = add_back_face(pioneer_card_counts_df)
pioneer_card_counts_df 

In [ ]:
for name in pioneer_card_counts_df['card_name']:
    if "Fable" in name:
        print(name)

In [ ]:
pioneer_card_counts_df = pioneer_card_counts_df.merge(cards_set_df, on = 'card_name', how = 'left')
pioneer_card_counts_df

In [ ]:
pioneer_card_counts_df = pioneer_card_counts_df.merge(set_year_df, on = 'set_name', how = 'left', copy = True)
pioneer_card_counts_df

In [ ]:
plt = ptd.create_base_graph(pioneer_card_counts_df, (9/255, 121/255, 105/255))
plt.axvline(x=19.2, color='blue', linestyle='--')
plt.text(16, 70000, 'Pioneer Legal', color = 'black', fontsize = 10)
plt.axvline(x=25.5, color='black', linestyle='--')
plt.text(22.7, 70000, 'Fire Design', color = 'black', fontsize = 10)
plt.show()

## Legacy

In [ ]:
base_path: str = os.path.join(DATA_DIRECTORY, '2023')
legacy_card_counts_df, legacy_player_counts_df = ltd.load_format_card_counts(base_path, 'legacy')
legacy_card_counts_df

In [ ]:
legacy_card_counts_df = add_back_face(legacy_card_counts_df)
legacy_card_counts_df

In [ ]:
for name in legacy_card_counts_df['card_name']:
    if "Fable" in name:
        print(name)

In [ ]:
legacy_card_counts_df = legacy_card_counts_df.merge(cards_set_df, on = 'card_name', how = 'left')
legacy_card_counts_df

In [ ]:
legacy_card_counts_df = legacy_card_counts_df.merge(set_year_df, on = 'set_name', how = 'left', copy = True)
legacy_card_counts_df

In [ ]:
plt = ptd.create_base_graph(legacy_card_counts_df, (255/255, 215/255, 0/255))
plt.axvline(x=25.5, color='black', linestyle='--')
plt.text(22.7, 30000, 'Fire Design', color = 'black', fontsize = 10)
plt.axvline(x=28, color='purple', linestyle='--')
plt.text(28.4, 35000, 'MH2', color = 'black', fontsize = 10)
plt.show()
